In [109]:
from datetime import datetime
import json
from kafka import KafkaConsumer

In [110]:
consumer=KafkaConsumer(bootstrap_servers='185.236.37.254:9092')
consumer.subscribe('preProcessedData')

In [85]:
from cassandra.cluster import Cluster
clstr=Cluster()
session=clstr.connect()

In [86]:
qry_create_keyspace = '''
CREATE KEYSPACE "keystore"
WITH replication = {'class': 'SimpleStrategy', 'replication_factor' : '2'};
'''
session.execute(qry_create_keyspace)

In [96]:
session = clstr.connect('keystore')

In [182]:
qry_create_table_posts = '''
CREATE TABLE IF NOT EXISTS posts(
   id TEXT,
   sender TEXT,
   sendtime TIMESTAMP,
   day int,
   month int,
   hour int,
   PRIMARY KEY((day, month, hour), sendtime))
WITH CLUSTERING ORDER BY (sendtime desc); '''
session.execute(qry_create_table_posts)

In [183]:
qry_create_table_channel = '''
CREATE TABLE IF NOT EXISTS channel(
   id TEXT,
   sender TEXT,
   sendtime TIMESTAMP,
   PRIMARY KEY(sender, sendtime))
WITH CLUSTERING ORDER BY (sendtime desc); '''
session.execute(qry_create_table_channel)

In [184]:
qry_create_table_keyword = '''
CREATE TABLE IF NOT EXISTS keyword(
   id TEXT,
   keyword TEXT,
   sendtime TIMESTAMP,
   PRIMARY KEY(keyword, sendtime))
WITH CLUSTERING ORDER BY (sendtime desc); '''
session.execute(qry_create_table_keyword)

In [172]:
prepared_posts = session.prepare("""INSERT INTO posts (id, sender, sendtime, day, month, hour) VALUES (?,?,?,?,?,?)""")
prepared_channel = session.prepare("""INSERT INTO channel (id, sender, sendtime) VALUES (?,?,?)""")
prepared_keyword = session.prepare("""INSERT INTO keyword (id, keyword, sendtime) VALUES (?,?,?)""")

In [208]:
for msg in consumer:
    d=json.loads(msg.value.decode('utf-8'))

    id = ''
    sender = 'Undefiend'
    sendtime = datetime.now().replace(microsecond = 0)
    day = 0
    month = 0
    hour = 0

    if d['id'] != None :
        id = d['id']
    
    if d['send_time'] != None :
        day = datetime.fromtimestamp(d['send_time']).day
        month = datetime.fromtimestamp(d['send_time']).month
        hour = datetime.fromtimestamp(d['send_time']).hour
        sendtime = datetime.fromtimestamp(d['send_time']).replace(microsecond=0)

    if d['sender_username'] != None :
        sender = d['sender_username']

    session.execute(prepared_posts, [id, sender, sendtime, day, month, hour])
    session.execute(prepared_channel, [id, sender, sendtime])
    if len(d['keywords']) > 0 :
        for i in d['keywords']:
            session.execute(prepared_keyword, [id, i, sendtime])
    
    if len(d['hashtags']) > 0 :
        for i in d['hashtags']:
            session.execute(prepared_keyword, [id, i, sendtime])        